In [ ]:
import tweepy
from pymongo import MongoClient
from tweepy import Stream
from tweepy.streaming import StreamListener
import os
import datetime

In [ ]:
dbUrl = os.getenv('MLAB_DB')
client = MongoClient(dbUrl)
db = client['mining']

In [ ]:
class StreamListener(StreamListener):
    def on_status(self, status):
        if 'python' in status.text.lower():
            insert('Python')
        elif 'golang' in status.text.lower():
            insert('Golang')
        elif 'javascript' in status.text.lower():
            insert('Javascript')
        else:
            print(status.text)
            
    def on_error(self, status_code):
        return False

In [ ]:
def insert(lang):
    tweets = db['tweets']
    id = tweets.insert_one({
        "lang": lang, 
        "date": datetime.datetime.utcnow()
    })

In [ ]:
def get_tweepy_client():
    consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
    consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
    access_token = os.getenv('TWITTER_ACCESS_TOKEN')
    access_secret = os.getenv('TWITTER_ACCESS_SECRET')

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    l = StreamListener()
    stream = Stream(auth, listener=l)
    return stream

In [ ]:
stream = get_tweepy_client()

In [7]:
stream.filter(languages=["en"], track=['python', 'golang', 'javascript'], async=True)